# SQLDatabase 工具包

这将帮助您开始使用 SQL Database [工具包](/docs/concepts/tools/#toolkits)。有关 `SQLDatabaseToolkit` 所有功能和配置的详细文档，请访问[API 参考](https://python.langchain.com/api_reference/community/agent_toolkits/langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit.html)。

`SQLDatabaseToolkit` 内的工具旨在与 `SQL` 数据库进行交互。

一个常见的应用是使智能体能够使用关系数据库中的数据来回答问题，可能以迭代的方式（例如，从错误中恢复）。

**⚠️ 安全提示 ⚠️**

构建 SQL 数据库的问答系统需要执行模型生成的 SQL 查询。这样做存在固有的风险。确保您的数据库连接权限始终针对您链/智能体的需求进行尽可能窄的范围界定。这将减轻但不能消除构建由模型驱动的系统的风险。有关更通用的安全最佳实践，[请参见此处](/docs/security)。

## 设置

为了启用对单个工具的自动跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

此工具包位于 `langchain-community` 包中：

In [ ]:
%pip install --upgrade --quiet  langchain-community

为了演示目的，我们将访问 LangChain [Hub](https://smith.langchain.com/hub) 中的一个提示。我们还将需要 `langgraph` 来演示如何将工具包与代理一起使用。使用工具包并非必须需要它。

In [ ]:
%pip install --upgrade --quiet langchainhub langgraph

## 实例化

`SQLDatabaseToolkit` 工具包需要：

- 一个 [SQLDatabase](https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.sql_database.SQLDatabase.html) 对象；
- 一个 LLM 或聊天模型（用于实例化 [QuerySQLCheckerTool](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.sql_database.tool.QuerySQLCheckerTool.html) 工具）。

下面，我们将使用这些对象来实例化工具包。首先，我们创建一个数据库对象。

本指南基于[这些说明](https://database.guide/2-sample-databases-sqlite/)，使用了 `Chinook` 示例数据库。

下面我们将使用 `requests` 库来获取 `.sql` 文件并创建一个内存中的 SQLite 数据库。请注意，这种方法很轻便，但它是临时的且不是线程安全的。如果愿意，您可以按照说明将文件本地保存为 `Chinook.db` 并通过 `db = SQLDatabase.from_uri("sqlite:///Chinook.db")` 来实例化数据库。

In [1]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_chinook_db()

db = SQLDatabase(engine)

我们还需要一个 LLM 或聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

现在我们可以实例化这个工具包：

In [3]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

## 工具

查看可用工具：

In [4]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x103d5fa60>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x103d5fa60>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x103d5fa60>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

您可以直接使用各个工具：

In [ ]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDatabaseTool,
)

## 在 Agent 中使用

遵循 [SQL Q&A 教程](/docs/tutorials/sql_qa/#agents)，下面我们为简单的问答 Agent 配备工具包中的工具。首先我们提取一个相关的 Prompt 并用其所需的参数进行填充：

In [6]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [7]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

然后我们实例化代理：

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, toolkit.get_tools(), prompt=system_message)

并向其发出查询：

In [9]:
example_query = "Which country's customers spent the most?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Which country's customers spent the most?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_EBPjyfzqXzFutDn8BklYACLj)
 Call ID: call_EBPjyfzqXzFutDn8BklYACLj
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_kGcnKpxRVFIY8dPjYIJbRoVU)
 Call ID: call_kGcnKpxRVFIY8dPjYIJbRoVU
  Args:
    table_names: Customer, Invoice, InvoiceLine
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL

我们还可以观察到代理从错误中恢复：

In [10]:
example_query = "Who are the top 3 best selling artists?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who are the top 3 best selling artists?
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_xAkvYiRFM7nCMKXsDNvk1OMx)
 Call ID: call_xAkvYiRFM7nCMKXsDNvk1OMx
  Args:
    query: SELECT artist_name, SUM(quantity) AS total_sold FROM sales GROUP BY artist_name ORDER BY total_sold DESC LIMIT 3
================================= Tool Message =================================
Name: sql_db_query

Error: (sqlite3.OperationalError) no such table: sales
[SQL: SELECT artist_name, SUM(quantity) AS total_sold FROM sales GROUP BY artist_name ORDER BY total_sold DESC LIMIT 3]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_K4Zvbowsq7XPgGFepbvc5G7i)
 Call ID: call_K4Zvbowsq7XPgGFepbvc5G7i
  Args:
================================= T

## 特定功能

`SQLDatabaseToolkit` 实现了一个 `.get_context` 方法，方便在提示或其他上下文中使用。

**⚠️ 免责声明 ⚠️**：代理可能会生成插入/更新/删除查询。当不需要这样做时，请使用自定义提示或创建没有写入权限的 SQL 用户。

最终用户可能会通过提出一个简单的问题，例如“运行尽可能大的查询”，来使您的 SQL 数据库过载。生成的查询可能如下所示：

```sql
SELECT * FROM "public"."users"
    JOIN "public"."user_permissions" ON "public"."users".id = "public"."user_permissions".user_id
    JOIN "public"."projects" ON "public"."users".id = "public"."projects".user_id
    JOIN "public"."events" ON "public"."projects".id = "public"."events".project_id;
```

对于事务性 SQL 数据库，如果上述某个表包含数百万行，该查询可能会给使用相同数据库的其他应用程序带来麻烦。

大多数面向数据仓库的数据库都支持用户级别配额，用于限制资源使用。

## API 参考

有关 SQLDatabaseToolkit 的所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/community/agent_toolkits/langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit.html)。